# Browse one file

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from dsgrid.dataformat.datafile import Datafile
from dsgrid.dataformat.datatable import Datatable

from ntbkhelp import OptionPresenter, show_enum, show_elements_with_data

## Choose data location

Review the choices in the first cell, select the path you want to use in the second cell, and run both cells.

In [2]:
dsgrid_nrel_base_path_windows = Path("//nrelnas01/PLEXOS/Projects/Load/dsgrid_v0.2.0/")
dsgrid_nrel_base_path_mac = Path("/Volumes/PLEXOS/Projects/Load/dsgrid_v0.2.0/")
dsgrid_oedi_base_path = None

In [3]:
dsgrid_base_path = dsgrid_nrel_base_path_windows

## Choose data set

Select whether you want to look at county-level or state-level data in the second cell. Run both cells.

In [4]:
dsgrid_county_dataset_path = dsgrid_base_path / "products" / "raw_complete"
dsgrid_state_dataset_path = dsgrid_base_path / "products" / "state_hourly_residuals"

In [5]:
dsgrid_dataset_path = dsgrid_county_dataset_path

## Specify a local directory for any outputs

Review and edit the specified path. (The default option should be available for most everyone, but feel to change as you see fit.) Then run the cell.

In [6]:
output_dir = Path.home() / "Documents" / "dsgrid-legacy-efs"

if not output_dir.exists():
    input_str = input(f"{output_dir!r} does not exist. Would you like to create it? [Y/n] ")
    if input_str[0].lower() == "y":
        output_dir.mkdir()
        print(f"Created {output_dir!r}.")

## Choose a file

In [7]:
files = [p.name for p in dsgrid_dataset_path.glob("*.dsg")]
options = OptionPresenter(files)
options.present_options()
input_str = input("Which file would you like to examine (input number)? ")
datafile = Datafile.load(dsgrid_dataset_path / options.get_option(input_str))

  1: chp_dg.dsg
  2: commercial.dsg
  3: commercial_gaps.dsg
  4: distributedpv_sectoral.dsg
  5: eia_annual_energy_by_sector.dsg
  6: historical_hourly_load.dsg
  7: industrial.dsg
  8: industrial_gaps.dsg
  9: loss_model.dsg
 10: municipal_water.dsg
 11: outdoor_lighting.dsg
 12: residential.dsg
 13: residential_gaps.dsg
 14: trans_rail_hourly.dsg
Which file would you like to examine (input number)? 7


## Examine Basic Information

In [8]:
show_enum(datafile.sector_enum)

industrial_sectors


,id,name
0,3111,Animal Food Manufacturing
1,3112,Grain and Oilseed Milling
2,3113,Sugar and Confectionery Product Manufacturing
3,3114,Fruit and Vegetable Preserving and Specialty F...
4,3115,Dairy Product Manufacturing
...,...,...
81,3371,Household and Institutional Furniture and Kitc...
82,3372,Office Furniture (including Fixtures) Manufact...
83,3379,Other Furniture Related Product Manufacturing
84,3391,Medical Equipment and Supplies Manufacturing


In [9]:
show_elements_with_data(datafile.sector_enum, list(datafile.keys()))

Subset of industrial_sectors with data:


,id,name
0,3111,Animal Food Manufacturing
1,3112,Grain and Oilseed Milling
2,3113,Sugar and Confectionery Product Manufacturing
3,3114,Fruit and Vegetable Preserving and Specialty F...
4,3115,Dairy Product Manufacturing
...,...,...
81,3371,Household and Institutional Furniture and Kitc...
82,3372,Office Furniture (including Fixtures) Manufact...
83,3379,Other Furniture Related Product Manufacturing
84,3391,Medical Equipment and Supplies Manufacturing


In [10]:
show_enum(datafile.geo_enum)

counties


,id,name
0,01001,"Autauga County, AL"
1,01003,"Baldwin County, AL"
2,01005,"Barbour County, AL"
3,01007,"Bibb County, AL"
4,01009,"Blount County, AL"
...,...,...
3229,72151,"Yabucoa Municipio, PR"
3230,72153,"Yauco Municipio, PR"
3231,78010,"St. Croix Island, VI"
3232,78020,"St. John Island, VI"


⚠️ **WARNING** ⚠️ The next cell may take a few minutes to run if the geography is counties

In [ ]:
geo_ids = []
for _id in datafile.geo_enum.ids:
    for sector_id in datafile:
        if datafile[sector_id].has_data(_id):
            geo_ids.append(_id)
            break
show_elements_with_data(datafile.geo_enum, geo_ids)

In [11]:
show_enum(datafile.enduse_enum)

industrial_enduses


,id,name,fuel,units
0,conventional_boiler_use,Conventional Boiler Use,Electricity,MWh
1,process_heating,Process Heating,Electricity,MWh
2,process_cooling_and_refrigeration,Process Cooling And Refrigeration,Electricity,MWh
3,machine_drive,Machine Drive,Electricity,MWh
4,electro_chemical_processes,Electro Chemical Processes,Electricity,MWh
5,other_process_use,Other Process Use,Electricity,MWh
6,facility_hvac,Facility Hvac,Electricity,MWh
7,facility_lighting,Facility Lighting,Electricity,MWh
8,other_facility_support,Other Facility Support,Electricity,MWh
9,onsite_transportation,Onsite Transportation,Electricity,MWh


In [12]:
for sector_id in datafile:
    df, _geo_ids, _scaling_factors = datafile[sector_id].get_data(0)
    break
show_elements_with_data(datafile.enduse_enum, list(df.columns))

Subset of industrial_enduses with data:


,id,name
0,conventional_boiler_use,Conventional Boiler Use
1,process_heating,Process Heating
2,process_cooling_and_refrigeration,Process Cooling And Refrigeration
3,machine_drive,Machine Drive
4,electro_chemical_processes,Electro Chemical Processes
5,other_process_use,Other Process Use
6,facility_hvac,Facility Hvac
7,facility_lighting,Facility Lighting
8,other_facility_support,Other Facility Support
9,onsite_transportation,Onsite Transportation


In [13]:
show_enum(datafile.time_enum)

standard_2012_hourly


,id,name
0,2012-01-01 01:00:00-05:00,2012-01-01 01:00:00-05:00
1,2012-01-01 02:00:00-05:00,2012-01-01 02:00:00-05:00
2,2012-01-01 03:00:00-05:00,2012-01-01 03:00:00-05:00
3,2012-01-01 04:00:00-05:00,2012-01-01 04:00:00-05:00
4,2012-01-01 05:00:00-05:00,2012-01-01 05:00:00-05:00
...,...,...
8779,2012-12-31 20:00:00-05:00,2012-12-31 20:00:00-05:00
8780,2012-12-31 21:00:00-05:00,2012-12-31 21:00:00-05:00
8781,2012-12-31 22:00:00-05:00,2012-12-31 22:00:00-05:00
8782,2012-12-31 23:00:00-05:00,2012-12-31 23:00:00-05:00


## Get Total Energy per Sector and Enduse with Datatable

⚠️ **WARNING** ⚠️ This cell may be slow and/or run out of memory for large files / county level data (e.g., commercial.dsg, residential.dsg).

In [14]:
data_series = Datatable(datafile).data
total_energy = data_series.reset_index().pivot_table(values=0, index="sector", columns="enduse", aggfunc=np.sum)
total_energy

KeyboardInterrupt: 

## Get Total Energy per Sector and Enduse by Iterating

This section runs for all files, but may take some time.

In [15]:
total_energy = []; index = []
for sector_id in datafile:
    sector_dataset = datafile[sector_id]
    sector_energy = None
    for i in range(sector_dataset.n_geos):
        df, geos, scaling_factors = sector_dataset.get_data(i)
        tmp = df.sum()
        for factor in scaling_factors:
            if sector_energy is None:
                sector_energy = (tmp * factor)
            else:
                sector_energy += (tmp * factor)
    if sector_energy is not None:
        total_energy.append(sector_energy.to_frame().T)
        index.append(sector_id)
total_energy = pd.concat(total_energy)
total_energy.index = index
total_energy

,conventional_boiler_use,process_heating,process_cooling_and_refrigeration,machine_drive,electro_chemical_processes,other_process_use,facility_hvac,facility_lighting,other_facility_support,onsite_transportation,other_nonprocess_use,end_use_not_reported
3111,184092.296875,253902.984375,1.570285e+06,2.555424e+06,13351.409180,108460.648438,5.010034e+05,440464.187500,120162.828125,55447.597656,25132.076172,132728.765625
3112,185956.031250,261401.750000,1.586180e+06,2.642632e+06,13486.576172,109558.562500,5.094328e+05,448720.093750,121379.164062,56008.945312,25386.500000,134072.328125
3113,88545.203125,122829.460938,7.552791e+05,1.237908e+06,6421.793945,52167.589844,2.414548e+05,212399.750000,57796.195312,26669.341797,12088.086914,63840.175781
3114,295652.281250,411468.937500,2.521872e+06,4.150072e+06,21442.361328,174187.453125,8.071319e+05,710235.937500,192981.265625,89048.781250,40362.082031,213162.062500
3115,354078.812500,501662.843750,3.020245e+06,5.080731e+06,25679.785156,208610.296875,9.726865e+05,857434.250000,231117.890625,106646.601562,48338.386719,255287.343750
...,...,...,...,...,...,...,...,...,...,...,...,...
3371,47809.070312,145743.640625,6.644653e+04,1.962124e+06,14585.807617,0.000000,8.059471e+05,666734.500000,133703.265625,43757.527344,18637.417969,0.000000
3372,12399.233398,37279.171875,1.723285e+04,5.006124e+05,3782.818115,0.000000,2.084639e+05,172226.546875,34675.851562,11348.453125,4833.604004,0.000000
3379,3209.796143,10053.569336,4.461072e+03,1.362211e+05,979.259277,0.000000,5.460832e+04,45296.828125,8976.547852,2937.778564,1251.275879,0.000000
3391,47539.414062,348710.000000,2.124102e+05,9.918645e+05,20903.871094,56305.574219,7.221358e+05,416576.437500,144978.453125,17195.095703,7080.342285,0.000000


In [16]:
total_energy.sum().sum() / 1.0E6

893.346368

## Get Total Energy per Sector and Enduse by Aggregating and Converting Units

Review user input in the first cell before running this section.

As with the previous section, running this code may take some time.

In [25]:
# Applying a map requires making a new HDF5 file on disk
overwrite = False     # Choices: True, False
energy_units = "TWh"  # Choices: kWh, MWh, GWh, TWh

In [26]:
from dsgrid.dataformat.dimmap import FilterToSingleFuelMap, FullAggregationMap, UnitConversionMap
from dsgrid.dataformat.enumeration import MultiFuelEndUseEnumeration, allsectors, conus, allenduses, annual

mappings = [
    ('-annual', FullAggregationMap(datafile.time_enum, annual)),
    ('-conus', FullAggregationMap(datafile.geo_enum, conus)),
    ('-allsectors', FullAggregationMap(datafile.sector_enum, allsectors)),    
]

# handle multi-fuel enduse enums
enduse_enum = datafile.enduse_enum
if isinstance(enduse_enum, MultiFuelEndUseEnumeration):
    mappings.append(f'-electricity', FilterToSingleFuelMap(enduse_enum, 'Electricity'))
    enduse_enum = mappings[-1][1].to_enum

# handle different electricity units
allenduses_units = allenduses.units(allenduses.ids[0])
if not (enduse_enum.units(enduse_enum.ids[0]) == allenduses_units):
    mappings.append(f'-{allenduses_units}', UnitConversionMap(
        enduse_enum, 
        [enduse_enum.units(enduse_enum.ids[0])], 
        [allenduses_units]))
    enduse_enum = mappings[-1][1].to_enum

mappings.extend([
    ('-allenduses', FullAggregationMap(enduse_enum, allenduses)),
    (f'-{energy_units}', UnitConversionMap(allenduses, [allenduses.units(allenduses.ids[0])], [energy_units]))
])

new_datafile = datafile
for postfix, mapping in mappings:
    new_path = output_dir / (new_datafile.filepath.stem + postfix + new_datafile.filepath.suffix)
    if new_path.exists() and overwrite:
        new_path.unlink()
    if new_path.exists():
        new_datafile = Datafile.load(new_path)
        continue
    print(f"Applying {mapping} and writing to {new_path}\n")
    new_datafile = new_datafile.map_dimension(new_path, mapping)
    
total_energy = Datatable(new_datafile).data.sum()
print(f"{datafile.filepath.name} describes {total_energy} {energy_units} of electricity")

industrial.dsg describes 893.3463134765625 TWh of electricity
